**IPython Notebook: To Preprocess the data for identifying the Job Sources and storing the data to the DataBase.**


In [ ]:
import pandas as pd                 # TO HANDLE DATAFRAMES
import collections                  
import sqlite3                      # DATABASE TO STORE DATA AND FOR FUTURE QUERYING
from urllib.parse import urlparse   # PARSE DOMAIN NAME FROM THE URL
import json

In [ ]:
jobs_path = '/content/drive/MyDrive/Pathrise-Assignment/Files/job_opportunities.csv'
jobBoards_path = '/content/drive/MyDrive/Pathrise-Assignment/Files/jobBoards.json'

In [ ]:
# df ->  DATAFRAME THAT HANDLES THE JOB OPPORTUNITIES DATA
df = pd.read_csv(jobs_path)
df.head()

,ID (primary key),Job Title,Company Name,Job URL
0,125114,Junior Software Engineer,talend,https://www.linkedin.com/jobs/view/2357043904/...
1,125115,Junior Software Engineer,Talend,https://jobs.lever.co/talend/d6f028a9-cafd-42f...
2,125116,Junior Software Engineer,Talend,https://jobs.lever.co/talend/d6f028a9-cafd-42f...
3,125117,NaN,Unknown,https://hopjump.com/jobs.html?2398218?gh_jid=2...
4,110964,NaN,Unknown,https://mail.google.com/mail/u/1/#inbox


In [ ]:
# RENAME THE COLUMN NAMES IN A FORMAT SIMILAR TO THE COLUMN NAMES IN THE DATABASE (REMOVING SPACES)
df = df.rename(columns={"ID (primary key)":"id",'Job Title':'job_title','Company Name':'company_name','Job URL':'job_url'})
df['job_source']= None
df.head()

,id,job_title,company_name,job_url,job_source
0,125114,Junior Software Engineer,talend,https://www.linkedin.com/jobs/view/2357043904/...,None
1,125115,Junior Software Engineer,Talend,https://jobs.lever.co/talend/d6f028a9-cafd-42f...,None
2,125116,Junior Software Engineer,Talend,https://jobs.lever.co/talend/d6f028a9-cafd-42f...,None
3,125117,NaN,Unknown,https://hopjump.com/jobs.html?2398218?gh_jid=2...,None
4,110964,NaN,Unknown,https://mail.google.com/mail/u/1/#inbox,None


In [ ]:
# READ JSON FILE USING PANDAS
job_boards_JSON = pd.read_json(jobBoards_path,orient='columns')
job_boards_JSON.head()

,job_boards
0,"{'name': 'Google', 'rating': 'Great', 'root_do..."
1,"{'name': 'Glassdoor', 'rating': 'Great', 'root..."
2,"{'name': 'AngelList', 'rating': 'Great', 'root..."
3,"{'name': 'LinkedIn', 'rating': 'Great', 'root_..."
4,"{'name': 'Dribble', 'rating': 'Great', 'root_d..."


In [ ]:
# job_boards_DF -> DATAFRAME TO HANDLE JOB BOARDS DATA
job_boards_DF = pd.read_json((job_boards_JSON['job_boards']).to_json(),orient='index')
job_boards_DF.head()

,name,rating,root_domain,logo_file,description
0,Google,Great,google.com,https://storage.googleapis.com/pathrise-app/jo...,Surprisingly good search tool for aggregrating...
1,Glassdoor,Great,glassdoor.com,https://storage.googleapis.com/pathrise-app/jo...,Often has different listings then Indeed. Also...
2,AngelList,Great,angel.co,https://storage.googleapis.com/pathrise-app/jo...,"A job board for startups. Quality varies, but ..."
3,LinkedIn,Great,linkedin.com,https://storage.googleapis.com/pathrise-app/jo...,There are great jobs on LinkedIn. Applicants o...
4,Dribble,Great,dribbble.com,https://storage.googleapis.com/pathrise-app/jo...,A job board specific for designers with qualit...


In [ ]:
# STORING THE DOMAIN NAME AND COMPANY NAME IN A HASHMAP TO FURTHER PARSE JOB URL AND IDENTIFY THE SOURCE.
_map = collections.defaultdict(str)
for name,domain in zip(job_boards_DF['name'],job_boards_DF['root_domain']):
  _map[domain] = name

In [ ]:
_map

defaultdict(str,
            {'angel.co': 'AngelList',
             'dribbble.com': 'Dribble',
             'employbl.com': 'Employbl',
             'gamasutra.com': 'Gamasutra',
             'github.com': 'Github',
             'glassdoor.com': 'Glassdoor',
             'google.com': 'Google',
             'governmentjobs.com': 'Government Jobs',
             'greenhouse.io': 'Greenhouse',
             'hired.com': 'Hired',
             'hiretechladies.com': 'Tech Ladies',
             'huntr.co': 'Huntr Jobs',
             'indeed.com': 'Indeed',
             'intern.supply': 'Intern Supply',
             'jobvite.com': 'Jobvite',
             'jopwell.com': 'Jopwell',
             'lever.co': 'Lever',
             'linkedin.com': 'LinkedIn',
             'monster.com': 'Monster',
             'simplyhired.com': 'SimplyHired',
             'smartrecruiters.com': 'SmartRecruiters',
             'stackoverflow.com': 'Stackoverflow',
             'stella.ai': 'Stella',
             'tri

In [ ]:
# CHECK IF DOMAIN NAME IN THE JOB URL IS PRESENT IN THE HASHMAP _map AS A KEY
#  : IF PRESENT, SET THE JOB SOURCE AS THE VALUE OF HASHMAP _map
#  : ELSE SET IT AS THE COMPANY NAME OR UNKNOWN IF UNAVAILABLE

for idx,url in enumerate(df['job_url']):
  try:
    domain = urlparse(url).netloc
    for root_domain,name in _map.items():
      if root_domain in domain:
        df.iloc[idx,df.columns.get_loc('job_source')] = name
        break
    if not df.iloc[idx]['job_source']:
      if df.iloc[idx]['company_name'] and df.iloc[idx]['company_name'] != 'Unknown':
        df.iloc[idx,df.columns.get_loc('job_source')] = df.iloc[idx,df.columns.get_loc('company_name')]
      else:
        df.iloc[idx,df.columns.get_loc('job_source')] = 'Unknown'
  except:
    continue

In [ ]:
# FIND THE COMPANIES THAT DOES NOT HAVE A JOB SOURCE AND HAS JOB SOURCE VALUE AS THE COMPANY NAME ITSELF
remaining_companies = set(df['job_source']).difference(set(job_boards_DF['name']))

In [ ]:
# CROSS CHECKING FOR VALIDATION
if 'LinkedIn' in remaining_companies:
  print(True)
else:
  print(False)

False


In [ ]:
# APPEND THE REMAINING COMPANIES IN THE job_boards_DF WITH name AS THE company name 
# AND DEFAULT VALUES FOR OTHER FIELDS AS BELOW
for company in remaining_companies:
  job_boards_DF = job_boards_DF.append({'name': company,'rating':'rating N/A','root_domain': None,'logo_file':'N/A','description':company}, ignore_index=True)

In [ ]:
job_boards_DF.tail()

,name,rating,root_domain,logo_file,description
1813,Republic,rating N/A,None,N/A,Republic
1814,Mark43,rating N/A,None,N/A,Mark43
1815,Glaxo Group,rating N/A,None,N/A,Glaxo Group
1816,"Go To Market Enablement - Associate / Manager,...",rating N/A,None,N/A,"Go To Market Enablement - Associate / Manager,..."
1817,Capitol Group,rating N/A,None,N/A,Capitol Group


In [ ]:
# TO COUNT THE TOTAL NUMBER OF JOBS FOR EACH JOB SOURCE
jobCounter = collections.defaultdict(int)
for source in df['job_source']:
  if source:
    jobCounter[source] += 1

In [ ]:
# DUMPING THE COUNTER IN A JSON FILE
with open("/content/drive/MyDrive/Pathrise-Assignment/Files/jobCounter.json", "w") as outfile:  
    json.dump(jobCounter, outfile)

In [ ]:
# PATH THE SQLITE3 DATABASE I CREATED USING TERMINAL
db_path = '/content/drive/MyDrive/Pathrise-Assignment/SQL_DB/jobOpportunites.db'

In [ ]:
# CONNECTION OBJECT TO THE DATABASE
con = sqlite3.connect(db_path)

In [ ]:
# DUMPING THE DATAFRAME AS TABLES DIRECTLY INTO THE DATABASE USING PANDAS' to_sql() FUNCTION.
df.to_sql('JobListings',con,if_exists='replace')
job_boards_DF.to_sql('JobBoards',con,if_exists='replace')

In [ ]:
curr = con.cursor()
for row in curr.execute('SELECT * FROM JobListings LIMIT 5'):
  print(row)

(0, 125114, 'Junior Software Engineer', 'talend', 'https://www.linkedin.com/jobs/view/2357043904/?refId=ZKEVDnVCmI5kUntfCsC7OQ%3D%3D&eBP=NotAvailableFromMidTier', 'LinkedIn')
(1, 125115, 'Junior Software Engineer', 'Talend', 'https://jobs.lever.co/talend/d6f028a9-cafd-42fe-9f90-f89f7cb695d9', 'Lever')
(2, 125116, 'Junior Software Engineer', 'Talend', 'https://jobs.lever.co/talend/d6f028a9-cafd-42fe-9f90-f89f7cb695d9/apply', 'Lever')
(3, 125117, None, 'Unknown', 'https://hopjump.com/jobs.html?2398218?gh_jid=2398218#positions', 'Unknown')
(4, 110964, None, 'Unknown', 'https://mail.google.com/mail/u/1/#inbox', 'Google')


In [ ]:
curr = con.cursor()
for row in curr.execute('SELECT * FROM JobBoards LIMIT 5'):
  print(row)

(0, 'Google', 'Great', 'google.com', 'https://storage.googleapis.com/pathrise-app/job_sources/google.png', 'Surprisingly good search tool for aggregrating jobs across other job boards. try to limit to only more recent jobs like posted in the past few weeks.')
(1, 'Glassdoor', 'Great', 'glassdoor.com', 'https://storage.googleapis.com/pathrise-app/job_sources/glassdoor.png', 'Often has different listings then Indeed. Also try to look under more unconventional locations if you can.')
(2, 'AngelList', 'Great', 'angel.co', 'https://storage.googleapis.com/pathrise-app/job_sources/angellist.png', 'A job board for startups. Quality varies, but several of our fellows have found success applying for jobs here.')
(3, 'LinkedIn', 'Great', 'linkedin.com', 'https://storage.googleapis.com/pathrise-app/job_sources/linkedin.png', 'There are great jobs on LinkedIn. Applicants on LinkedIn are often higher quality for recruiters, but come at a more expensive cost.')
(4, 'Dribble', 'Great', 'dribbble.com',

In [ ]:
# COMMITING AND CLOSING THE CONNECTION
con.commit()
con.close()